## **HW 4 Problems** ##

In [14]:
import numpy as np
import transforms as tf
import sympy as sp

**Problem 2**

Let R represent a **rotation of 90 about y_0** followed by a **rotation of 45 about z_1**.  
(a) Find the equivalent **axis/angle to represent R**. **Sketch** the initial and final frames and the equivalent axis vector r.   


In [12]:
R = np.eye(3)
R = R @ tf.roty(np.pi/2)  # 90 deg about y_0
R = R @ tf.rotz(np.pi/4)  # 45 deg about z_1
print("Rotation Matrix R:\n", R)

# (a) Axis/Angle Representation
axis_angle_data = tf.R2axis(R)

angle_rad = axis_angle_data[0]
axis = axis_angle_data[1:4]

print(f"\n(a) Axis/Angle Representation:")
print(f"Angle (radians): {angle_rad:.4f}")
print(f"Angle (degrees): {np.degrees(angle_rad):.2f}")

Rotation Matrix R:
 [[ 0.          0.          1.        ]
 [ 0.70710678  0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]]

(a) Axis/Angle Representation:
Angle (radians): 1.7178
Angle (degrees): 98.42


(b) Also find the **equivalent quaternion represntation**

In [13]:
quaternion = tf.R2quat(R)

print("Equivalent Quaternion Representation (nu, ex, ey, ez):\n", quaternion)

Equivalent Quaternion Representation (nu, ex, ey, ez):
 [0.65328148 0.27059805 0.65328148 0.27059805]


**Problem 3**

I strongly suggest **SymPy** for this problem. Using three variables to represent **yaw**, **pitch**, and **roll** angles, ($\psi$, $\theta$, $\phi$), do the following (assuming a rotation about z, then y, then x for current axis convention -- this woudl be identical to a rotation about x, then y, then z in a fixed frame convention):

(a) compute a rotation matrix (you may use code from the posted solution for HW02 for symbolic version of rotx, roty, and rotz functions).  

In [ ]:
sp.init_printing(use_unicode=True)

psi, theta, phi = sp.symbols('psi theta phi')

Rz = sp.Matrix([[sp.cos(psi), -sp.sin(psi), 0],
                [sp.sin(psi), sp.cos(psi), 0],  
                [0, 0, 1]])
Ry = sp.Matrix([[sp.cos(theta), 0, sp.sin(theta)],
                [0, 1, 0],
                [-sp.sin(theta), 0, sp.cos(theta)]])
Rx = sp.Matrix([[1, 0, 0],
                [0, sp.cos(phi), -sp.sin(phi)],
                [0, sp.sin(phi), sp.cos(phi)]])

(b) Use this to transfrom a unit vector in the z-direction.

In [ ]:
unit_Vector_z = sp.Matrix([0, 0, 1])
R_sym = Rz * Ry * Rx
unit_Vector_z_rotated = R_sym * unit_Vector_z

print("\nSymbolic Rotated Unit Vector along z-axis:")
unit_Vector_z_rotated

(c) Looking at the elements of the rotation matrix devise an algorithm to determine the angles ($\psi$, $\theta$, $\phi$). Hint -- find the pitch angle first.  

In [ ]:
# Algorithm to devise angles

def find_euler_angles(R):
    if abs(R[2,0]) != 1:
        theta1 = -np.arcsin(R[2,0])
        theta2 = np.pi - theta1

        psi1 = np.arctan2(R[2,1]/np.cos(theta1), R[2,2]/np.cos(theta1))
        psi2 = np.arctan2(R[2,1]/np.cos(theta2), R[2,2]/np.cos(theta2))

        phi1 = np.arctan2(R[1,0]/np.cos(theta1), R[0,0]/np.cos(theta1))
        phi2 = np.arctan2(R[1,0]/np.cos(theta2), R[0,0]/np.cos(theta2))

        return (psi1, theta1, phi1), (psi2, theta2, phi2)
    else:
        phi = 0
        if R[2,0] == -1:
            theta = np.pi/2
            psi = phi + np.arctan2(R[0,1], R[0,2])
        else:
            theta = -np.pi/2
            psi = -phi + np.arctan2(-R[0,1], -R[0,2])
        return (psi, theta, phi),

# Testing
theta = np.pi/4  # 45 degrees
phi = np.pi/6    # 30 degrees
psi = np.pi/3    # 60 degrees

R_test = tf.rotz(psi) @ tf.roty(theta) @ tf.rotx(phi)
angles = find_euler_angles(R_test)
print("\n(c) Computed Euler Angles (degrees):")
for angle_set in angles:
    # Convert angles to degrees for better readability
    print(f"psi: {np.degrees(angle_set[0]):.2f}, theta: {np.degrees(angle_set[1]):.2f}, phi: {np.degrees(angle_set[2]):.2f}")